In [1]:
import numpy as np
import networkx as nx
import networkx.algorithms.community as comm

In [2]:
def find_community(pop_dict, G, G_undirected):

  #Creating initial community
    c = []
    n = max(pop_dict, key=pop_dict.get)
#     print('Community centre:',n)
    c.append(n)
    for node in list(G_undirected.neighbors(n)):
        if node not in [node for community in all_communities for node in community]: ###########
            temp_set = set(list(G_undirected.neighbors(n)))
            temp_set.add(n)
            num = len(set(G_undirected.neighbors(node)).intersection(temp_set))
            den = len(list(G_undirected.neighbors(node)))
            b = (num+1)/(den+1)
            if b > 0.5:
                c.append(node)
    for x in c:
        if x in list(pop_dict.keys()):
            pop_dict.pop(x)

    return c,pop_dict

def expand_community(pop_dict, G, G_undirected,c):
    
    #Expanding the initial community
    N = []
    for node in c:
        for n in G_undirected.neighbors(node):
            if n not in c:
                N.append(n)
    N = list(set(N))
        
    for node in N:
        if node not in [val for i in all_communities for val in i]:
            num = len(set(list(G_undirected.neighbors(node))).intersection(set(c)))+1
            den = len(list(G_undirected.neighbors(node)))+1
            b = num/den 
            if b > 0.55:
                all_communities[-1].append(node)
                
            elif b >= 0.4:
                all_communities[-1].append(node)
                c = all_communities[-1]
                communities = [set(i) for i in all_communities]
                communities.append(set(pop_dict.keys())-(set(c)))
                if comm.modularity(G,communities) < mod_init:
                    all_communities[-1].remove(node)

    for x in [val for i in all_communities for val in i]:
        if x in list(pop_dict.keys()):
            pop_dict.pop(x)

    return pop_dict

In [3]:
def node_removal(i,j,all_communities):

    out_deg = list(G.neighbors(j))
    in_deg = list(set(G_undirected.neighbors(j))-set(G.neighbors(j)))
    n_out_comm = len(set(out_deg).intersection(set(i)))
    n_in_comm = len(set(in_deg).intersection(set(i)))
    n_out = len(out_deg)
    n_in = len(in_deg)
    if (n_out_comm==(n_in-n_in_comm)) and (n_in_comm==(n_out-n_out_comm)):
        return j
    return 'NAN'

In [4]:
G_undirected = nx.read_gml('karate.gml',label='id')

Data = open('karate.csv', "r")
next(Data, None)  # skip the first line in the input file
Graphtype = nx.DiGraph()

G = nx.parse_edgelist(Data, delimiter=',', create_using=Graphtype,nodetype=int)


attributes = {}
pop_dict = dict(G_undirected.degree())
all_communities=[]
na_community = []

while len(pop_dict)>0:
    
    c,pop_dict = find_community(pop_dict,G,G_undirected)
    all_communities.append(c)
    communities = [set(i) for i in all_communities]
    communities.append(set(pop_dict.keys())-(set(c)))
    mod_init = comm.modularity(G,communities)
    pop_dict = expand_community(pop_dict,G,G_undirected,c)
    
for idx,community in enumerate(all_communities):
    for node in community:
#         print(idx)
        attributes[node] = idx + 1

nx.set_node_attributes(G, attributes, name="comm")  
for i in range(len(all_communities)):
    for j in all_communities[i]:
        val = node_removal(all_communities[i],j,all_communities)
        if val==j:
            attributes[j] = 'NA'
            all_communities[i].remove(j)
            na_community.append(j)

all_communities.append(na_community)
nx.set_node_attributes(G, attributes, name="comm")    

print('\nFinal communities:',all_communities)


Final communities: [[34, 9, 15, 16, 19, 21, 23, 24, 27, 28, 29, 30, 31, 32, 33, 25, 26], [1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 18, 20, 22, 17], [10]]
